# Init for pyqcu.

In [1]:
import cupy as cp
import numpy as np
import functools
from pyqcu import define
from pyqcu import io
from pyqcu import qcu
from pyqcu import eigen, cg, bistabcg
from opt_einsum import contract
from pyqcu.set import params, argv, set_ptrs
params[define._NODE_RANK_] = define.rank
params[define._NODE_SIZE_] = define.size
kappa = 1 / (2 * argv[define._MASS_] + 8)
print('My rank is ', define.rank)
gauge_filename = f"quda_wilson-bistabcg-gauge_-{params[define._LAT_X_]}-{params[define._LAT_Y_]}-{params  [define._LAT_Z_]}-{params[define._LAT_T_]}-{params[define._LAT_XYZT_]}-{params[define._GRID_X_]}-{params[define._GRID_Y_]}-{params[define._GRID_Z_]}-{params[define._GRID_T_]}-{params[define._PARITY_]}-{params[define._NODE_RANK_]}-{params[define._NODE_SIZE_]}-{params[define._DAGGER_]}-f.h5"
print("Parameters:", params)


    @@@@@@######QCU NOTES START######@@@@@@@
    0. Required: MPI(e.g. 4.1.2), CUDA(e.g. 12.4), CMAKE(e.g. 3.22.1), GCC(e.g. 11.4.0), HDF5-MPI(e.g. 1.10.7,'apt install libhdf5-mpi-dev && export HDF5_MPI="ON" && pip install --no-binary=h5py h5py').
    1. The libqcu.so was compiled when pyqcu setup in download_path/PyQCU/lib, please add this path to your LD_LIBRARY_PATH.
    2. The QCU(PyQCU) splite grid by x->y->z->t, lattice by x->y->z->t->p->d->c->c or x->y->z->t->c->s(->p) and x->y->z->t->c->s->c->s(->p).
    3. The QUDA(PyQUDA) splite grid by t->z->y->x, lattice by c->c->x->y->z->t->p->d or c->s->x->y->z->t(->p) and c->s->c->s->x->y->z->t(->p).
    4. The QCU input params in numpy array(dtype=np.int32), argv in  numpy array(dtype=np.float32 or float64) array, set_ptrs in numpy array(dtype=np.int64), other in cupy array(dtype=cp.complex64 or complex128).
    5. The smallest lattice size is (x=4,y=4,z=4,t=8) that QCU support.
    @@@@@@######QCU NOTES END######@@@@@@@
    
Parameter

In [2]:
wilson_cg_params = params.copy()
wilson_cg_params[define._SET_INDEX_] = 0
wilson_cg_params[define._SET_PLAN_] = define._SET_PLAN1_
qcu.applyInitQcu(set_ptrs, wilson_cg_params, argv)

gridDim.x               :4096
blockDim.x              :128
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:0
host_params[_SET_PLAN_] :1
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :1024
lat_3dim[_YZT_]         :32768
lat_3dim[_XZT_]         :16384
lat_3dim[_XYT_]         :16384
lat_3dim[_XYZ_]         :16384
lat_4dim                :524288
grid_2dim[_XY_]         :1
grid_2dim[_XZ_]         :1
grid_2dim[_XT_]         :1
grid_2

In [3]:
wilson_dslash_eo_params = params.copy()
wilson_dslash_eo_params[define._SET_INDEX_] = 1
wilson_dslash_eo_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_eo_params[define._PARITY_] = define._EVEN_
wilson_dslash_eo_params[define._DAGGER_] = define._NO_USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_eo_params, argv)

gridDim.x               :4096
blockDim.x              :128
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:1
host_params[_SET_PLAN_] :0
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :1024
lat_3dim[_YZT_]         :32768
lat_3dim[_XZT_]         :16384
lat_3dim[_XYT_]         :16384
lat_3dim[_XYZ_]         :16384
lat_4dim                :524288
grid_2dim[_XY_]         :1
grid_2dim[_XZ_]         :1
grid_2dim[_XT_]         :1
grid_2

In [4]:
wilson_dslash_eo_dag_params = params.copy()
wilson_dslash_eo_dag_params[define._SET_INDEX_] = 2
wilson_dslash_eo_dag_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_eo_dag_params[define._PARITY_] = define._EVEN_
wilson_dslash_eo_dag_params[define._DAGGER_] = define._USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_eo_dag_params, argv)

gridDim.x               :4096
blockDim.x              :128
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :1
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:2
host_params[_SET_PLAN_] :0
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :1024
lat_3dim[_YZT_]         :32768
lat_3dim[_XZT_]         :16384
lat_3dim[_XYT_]         :16384
lat_3dim[_XYZ_]         :16384
lat_4dim                :524288
grid_2dim[_XY_]         :1
grid_2dim[_XZ_]         :1
grid_2dim[_XT_]         :1
grid_2

In [5]:
wilson_dslash_oe_params = params.copy()
wilson_dslash_oe_params[define._SET_INDEX_] = 3
wilson_dslash_oe_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_oe_params[define._PARITY_] = define._ODD_
wilson_dslash_oe_params[define._DAGGER_] = define._NO_USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_oe_params, argv)

gridDim.x               :4096
blockDim.x              :128
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :1
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:3
host_params[_SET_PLAN_] :0
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :1024
lat_3dim[_YZT_]         :32768
lat_3dim[_XZT_]         :16384
lat_3dim[_XYT_]         :16384
lat_3dim[_XYZ_]         :16384
lat_4dim                :524288
grid_2dim[_XY_]         :1
grid_2dim[_XZ_]         :1
grid_2dim[_XT_]         :1
grid_2

In [6]:
wilson_dslash_oe_dag_params = params.copy()
wilson_dslash_oe_dag_params[define._SET_INDEX_] = 4
wilson_dslash_oe_dag_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_oe_dag_params[define._PARITY_] = define._ODD_
wilson_dslash_oe_dag_params[define._DAGGER_] = define._USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_oe_dag_params, argv)

gridDim.x               :4096
blockDim.x              :128
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :1
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :1
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:4
host_params[_SET_PLAN_] :0
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :1024
lat_3dim[_YZT_]         :32768
lat_3dim[_XZT_]         :16384
lat_3dim[_XYT_]         :16384
lat_3dim[_XYZ_]         :16384
lat_4dim                :524288
grid_2dim[_XY_]         :1
grid_2dim[_XZ_]         :1
grid_2dim[_XT_]         :1
grid_2

In [7]:
print("Set pointers:", set_ptrs)
print("Set pointers data:", set_ptrs.data)

Set pointers: [93925773197920 93925782987136 93925783161264 93925783337792
 93925783476064              0              0              0
              0              0]
Set pointers data: <memory at 0x7fb6121d50c0>


# Read from hdf5 files.

In [8]:
print("Gauge filename:", gauge_filename)
gauge = io.hdf5_xxxtzyx2grid_xxxtzyx(params, gauge_filename)
fermion_in_filename = gauge_filename.replace("gauge", "fermion-in")
print("Fermion in filename:", fermion_in_filename)
fermion_in = io.hdf5_xxxtzyx2grid_xxxtzyx(
    params, fermion_in_filename)
fermion_out_filename = gauge_filename.replace("gauge", "fermion-out")
print("Fermion out filename:", fermion_out_filename)
quda_fermion_out = io.hdf5_xxxtzyx2grid_xxxtzyx(
    params, fermion_out_filename)
fermion_out = cp.zeros_like(fermion_in)
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
# eigenvalues_filename = gauge_filename.replace("gauge", "eigenvalues")
# print("Eigenvalues filename:", eigenvalues_filename)
# eigenvalues = io.hdf5_xxx2xxx(file_name=eigenvalues_filename)
# print("Eigenvalues data:", eigenvalues.data)
# print("Eigenvalues shape:", eigenvalues.shape)
# eigenvectors_filename = gauge_filename.replace("gauge", "eigenvectors")
# print("Eigenvectors filename:", eigenvectors_filename)
# eigenvectors = io.eigenvectors2esctzyx(
#     params=params, eigenvectors=io.hdf5_xxx2xxx(file_name=eigenvectors_filename))
# print("Eigenvectors data:", eigenvectors.data)
# print("Eigenvectors shape:", eigenvectors.shape)
orth_eigenvectors_filename = gauge_filename.replace("gauge", "orth_eigenvectors")
print("Orth orth_eigenvectors filename:", orth_eigenvectors_filename)
orth_eigenvectors = io.eigenvectors2esctzyx(
    params=params, eigenvectors=io.hdf5_xxx2xxx(file_name=orth_eigenvectors_filename))
print("Orth orth_eigenvectors data:", orth_eigenvectors.data)
print("Orth orth_eigenvectors shape:", orth_eigenvectors.shape)


Gauge filename: quda_wilson-bistabcg-gauge_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 32, Grid Lat X: 16
All Dest Shape: (3, 3, 4, 2, 32, 32, 32, 16)
Dest Shape: (3, 3, 4, 2, 32, 32, 32, 16)
Fermion in filename: quda_wilson-bistabcg-fermion-in_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 32, Grid Lat X: 16
All Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Fermion out filename: quda_wilson-bistabcg-fermion-out_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 32, Grid Lat X: 16
All Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Fermion out data: <MemoryPointer 0xb28400000 device=0 mem=<cupy.cuda.memory.PooledMemor

# Run wilson bistabcg from pyqcu test.

In [9]:
qcu.applyWilsonBistabCgQcu(fermion_out, fermion_in,
                           gauge, set_ptrs, wilson_cg_params)
# qcu.applyWilsonCgQcu(fermion_out, fermion_in,
#                            gauge, set_ptrs, wilson_cg_params)
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
print("QUDA Fermion out data:", quda_fermion_out.data)
print("QUDA Fermion out shape:", quda_fermion_out.shape)
print("Difference:", cp.linalg.norm(fermion_out -
      quda_fermion_out)/cp.linalg.norm(quda_fermion_out))

##RANK:0##LOOP:118##Residual:(2.27222e-10,1.97371e-23i)
multi-gpu wilson bistabcg total time: (without malloc free memcpy) :1.811516160 sec
######TIME  :4993.57######
##RANK      :0
##LOOP      :999
##tmp0      :(1.03257e-11,2.49512e-12i)
##tmp1      :(4.79284e-12,-2.12052e-23i)
##rho_prev  :(-2.31288e-06,4.83391e-06i)
##rho       :(-2.31288e-06,4.83391e-06i)
##alpha     :(0.629024,-0.434716i)
##beta      :(0.059529,-0.0243195i)
##omega     :(2.1544,0.520593i)
##send_tmp  :(0.00984323,0i)
##norm2_tmp :(4.97484e+07,0.000224118i)
##diff_tmp  :(1.9786e-10,-8.91365e-22i)
##lat_4dim  :(524288,0i)
Fermion out data: <MemoryPointer 0xb28400000 device=0 mem=<cupy.cuda.memory.PooledMemory object at 0x7fb681de7330>>
Fermion out shape: (2, 4, 3, 32, 32, 32, 16)
QUDA Fermion out data: <MemoryPointer 0xb22400000 device=0 mem=<cupy.cuda.memory.PooledMemory object at 0x7fb681da3ef0>>
QUDA Fermion out shape: (2, 4, 3, 32, 32, 32, 16)
Difference: 3.056118e-07


# Give CG & BISTABCG Dslash.
> src_o-set_ptr->kappa()**2*dslash_oe(dslash_eo(src_o))

In [10]:
def cg_dslash_no_dag(src):
    tmp0 = cp.zeros_like(src)
    tmp1 = cp.zeros_like(src)
    qcu.applyWilsonDslashQcu(
        tmp0, src, gauge, set_ptrs, wilson_dslash_eo_params)
    qcu.applyWilsonDslashQcu(
        tmp1, tmp0, gauge, set_ptrs, wilson_dslash_oe_params)
    return src-kappa**2*tmp1


def cg_dslash_dag(src):
    tmp0 = cp.zeros_like(src)
    tmp1 = cp.zeros_like(src)
    qcu.applyWilsonDslashQcu(
        tmp0, src, gauge, set_ptrs, wilson_dslash_eo_dag_params)
    qcu.applyWilsonDslashQcu(
        tmp1, tmp0, gauge, set_ptrs, wilson_dslash_oe_dag_params)
    return src-kappa**2*tmp1


def cg_dslash(src):
    return cg_dslash_dag(cg_dslash_no_dag(src))

def bistabcg_dslash(src):
    return cg_dslash_no_dag(src)



# Give matvec.

In [11]:
def matvec(src):
    return cg_dslash(src)

# Run matvec(eigenvector[.]) ?= eigenvalue[.]*eigenvector[.] for eigen test. (pass, don't run this)

In [12]:
# for i, ev in enumerate(eigenvalues):
#     print(f"λ_{i} = {ev:.2e}")
#     # Verify eigenvector
#     v = eigenvectors[i]
#     w = cp.zeros_like(v)
#     w = cg_dslash(v)
#     error = cp.linalg.norm(w - ev * v) / cp.linalg.norm(w)
#     print(f"Relative error: {error:.2e}")
#     j = i+1
#     if j == len(eigenvalues):
#         j = 0
#     print(
#         f"Diff between λ_{i} and λ_{j}: {cp.linalg.norm(eigenvectors[i] - eigenvectors[j])/cp.linalg.norm(eigenvectors[i]):.2e}")

# Give guage's eigenvalues and eigenvectors to hdf5 files. (pass, don't run this)

In [13]:
# eigen_solver = eigen.solver(
#     n=params[define._LAT_XYZT_] * define._LAT_HALF_SC_, k=define._LAT_Ne_,matvec=cg_dslash,dtype=gauge.dtype)
# eigenvalues, eigenvectors = eigen_solver.run()
# io.xxx2hdf5_xxx(
#     eigenvalues, params, gauge_filename.replace("gauge", "eigenvalues"))
# io.xxx2hdf5_xxx(
#     eigenvectors, params, gauge_filename.replace("gauge", "eigenvectors"))

# Origin CG. (pass, don't run this)

In [14]:
# b_e = fermion_in[define._EVEN_].flatten()
# b_o = fermion_in[define._ODD_].flatten()
# b__o = cp.zeros_like(b_o)
# tmp = cp.zeros_like(b_o)
# # b__o=b_o+kappa*D_oe(b_e)
# qcu.applyWilsonDslashQcu(tmp, b_e, gauge, set_ptrs, wilson_dslash_oe_params)
# b__o = b_o+kappa*tmp
# # b__o -> Dslash^dag b__o
# b__o = cg_dslash_dag(b__o)
# # Dslash(x_o)=b__o
# cg_solver = cg.slover(b=b__o, matvec=cg_dslash, tol=1e-10, max_iter=1000000)
# x_o = cg_solver.run()
# # x_e  =b_e+kappa*D_eo(x_o)
# qcu.applyWilsonDslashQcu(tmp, x_o, gauge, set_ptrs, wilson_dslash_eo_params)
# x_e = b_e+kappa*tmp
# # give qcu_fermion_out
# qcu_fermion_out = cp.zeros_like(quda_fermion_out)
# qcu_fermion_out[define._EVEN_] = x_e.reshape(
#     quda_fermion_out[define._EVEN_].shape)
# qcu_fermion_out[define._ODD_] = x_o.reshape(
#     quda_fermion_out[define._ODD_].shape)
# print(np.linalg.norm(qcu_fermion_out-quda_fermion_out) /
#       np.linalg.norm(quda_fermion_out))

# Origin BISTABCG. (pass, don't run this)

In [15]:
# b_e = fermion_in[define._EVEN_].flatten()
# b_o = fermion_in[define._ODD_].flatten()
# b__o = cp.zeros_like(b_o)
# tmp = cp.zeros_like(b_o)
# # b__o=b_o+kappa*D_oe(b_e)
# qcu.applyWilsonDslashQcu(tmp, b_e, gauge, set_ptrs, wilson_dslash_oe_params)
# b__o = b_o+kappa*tmp
# # Dslash(x_o)=b__o
# bistabcg_solver = bistabcg.slover(
#     b=b__o, matvec=bistabcg_dslash, tol=1e-10, max_iter=1000000)
# x_o = bistabcg_solver.run()
# # x_e  =b_e+kappa*D_eo(x_o)
# qcu.applyWilsonDslashQcu(tmp, x_o, gauge, set_ptrs, wilson_dslash_eo_params)
# x_e = b_e+kappa*tmp
# # give qcu_fermion_out
# qcu_fermion_out = cp.zeros_like(quda_fermion_out)
# qcu_fermion_out[define._EVEN_] = x_e.reshape(
#     quda_fermion_out[define._EVEN_].shape)
# qcu_fermion_out[define._ODD_] = x_o.reshape(
#     quda_fermion_out[define._ODD_].shape)
# print(np.linalg.norm(qcu_fermion_out-quda_fermion_out) / \
#     np.linalg.norm(quda_fermion_out))

# Give guage's orth_eigenvectors to hdf5 files. (pass, don't run this)

In [16]:
# _eigenvectors = io.xxxtzyx2mg_xxxtzyx(input_array=eigenvectors, params=params)
# _eigenvectors.shape  # escTtZzYyXx
# def orthogonalize(eigenvectors):
#     _eigenvectors = eigenvectors.copy()
#     size_e, size_s, size_c, size_T, size_t, size_Z, size_z, size_Y, size_y, size_X, size_x = eigenvectors.shape
#     print(size_e, size_s, size_c, size_T, size_t,
#           size_Z, size_z, size_Y, size_y, size_X, size_x)
#     for T in range(size_T):
#         for Z in range(size_Z):
#             for Y in range(size_Y):
#                 for X in range(size_X):
#                     origin_matrix = eigenvectors[:,
#                                                  :, :, T, :, Z, :, Y, :, X, :]
#                     _shape = origin_matrix.shape
#                     _origin_matrix = origin_matrix.reshape(size_e, -1)
#                     condition_number = np.linalg.cond(_origin_matrix.get())
#                     print(f"矩阵条件数: {condition_number}")
#                     a = _origin_matrix[:, 0]
#                     b = _origin_matrix[:, -1]
#                     print(cp.dot(a.conj(), b))
#                     Q = cp.linalg.qr(_origin_matrix.T)[0]
#                     condition_number = np.linalg.cond(Q.get())
#                     print(f"矩阵条件数: {condition_number}")
#                     a = Q[:, 0]
#                     b = Q[:, -1]
#                     print(cp.dot(a.conj(), b))
#                     _eigenvectors[:, :, :, T, :, Z, :, Y, :, X, :] = Q.T.reshape(
#                         _shape)
#     return _eigenvectors
# orth_eigenvectors = orthogonalize(_eigenvectors)
# io.xxx2hdf5_xxx(
#     orth_eigenvectors, params, gauge_filename.replace("gauge", "orth_eigenvectors"))

# MultiGrid - give grids.

In [17]:
testvectors = io.xxxtzyx2mg_xxxtzyx(input_array=orth_eigenvectors, params=params)
_src = io.xxxtzyx2mg_xxxtzyx(
    input_array=fermion_in[define._EVEN_], params=params)

Input Array Shape: (24, 4, 3, 32, 32, 32, 16)
Dest Shape: (24, 4, 3, 8, 4, 4, 8, 4, 8, 4, 4)
Input Array Shape: (4, 3, 32, 32, 32, 16)
Dest Shape: (4, 3, 8, 4, 4, 8, 4, 8, 4, 4)


# MultiGrid - R*vector.
![](./image0-dev40.png)

In [18]:
r_src = _src


def r_vec(src):
    return contract("escTtZzYyXx,scTtZzYyXx->eTZYX", testvectors, src)


r_dest = r_vec(r_src)

In [19]:
r_dest.shape

(24, 8, 4, 4, 4)

# MultiGrid - P*vector.
![](./image1-dev40.png)


In [20]:
p_src = r_dest


def p_vec(src):
    return contract("escTtZzYyXx,eTZYX->scTtZzYyXx", cp.conj(testvectors), src)


p_dest = p_vec(p_src)

In [21]:
p_dest.shape

(4, 3, 8, 4, 4, 8, 4, 8, 4, 4)

# MultiGrid - verify above.
![](./image2-dev40.png)

In [22]:
print(cp.linalg.norm(r_src))
print(cp.linalg.norm(p_dest))

3547.24
3531.9172


In [23]:
print(cp.linalg.norm(r_src-p_dest)/cp.linalg.norm(r_src))

0.09284756


In [24]:
print(cp.linalg.norm(r_src-p_vec(r_vec(r_src)))/cp.linalg.norm(r_src))

0.09284756


In [25]:
r_src.flatten()[:50]

array([1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j], dtype=complex64)

In [26]:
r_src.flatten()[-50:]

array([1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j], dtype=complex64)

In [27]:
p_dest.flatten()[:50]

array([1.1924026 +0.96220475j, 1.1314651 +0.9596404j ,
       1.0552521 +0.8234522j , 1.0549538 +0.92514884j,
       0.9615977 +1.0393802j , 0.95300674+1.0415285j ,
       0.9523357 +0.9495436j , 0.78334856+1.0426004j ,
       1.1427466 +1.1257019j , 0.8673905 +1.072619j  ,
       0.7877028 +1.033166j  , 0.81615794+0.8263445j ,
       0.7418158 +1.0979842j , 0.9804515 +0.89743346j,
       0.99102736+0.9627855j , 1.045865  +1.0467168j ,
       1.0703461 +1.0456425j , 0.90627563+0.97132605j,
       1.0373149 +1.0902432j , 0.82147443+1.0707421j ,
       0.86649394+0.89142334j, 0.95891976+0.98738074j,
       0.8901605 +1.1672864j , 0.79725903+0.9623244j ,
       1.0696645 +0.9613617j , 0.9215935 +1.0815051j ,
       0.9532187 +0.92094266j, 1.1357502 +1.0021203j ,
       0.85560083+1.1228731j , 0.78500533+1.1069124j ,
       0.92780966+0.97881925j, 1.028975  +1.0910051j ,
       1.0465842 +0.9475676j , 0.9770593 +1.115533j  ,
       0.74214095+1.0821452j , 1.2050415 +1.0633073j ,
       0.9

In [28]:
p_dest.flatten()[-50:]

array([1.015209  +0.9675929j , 0.9580551 +0.8241955j ,
       0.93845475+0.98816234j, 1.0123483 +0.95277333j,
       1.1093609 +1.1002198j , 0.9865519 +0.99884725j,
       0.8775487 +0.9521471j , 0.8883637 +0.94987065j,
       1.1569403 +0.9969489j , 1.0115447 +1.0523158j ,
       1.2233114 +0.9101221j , 1.0171493 +1.0560522j ,
       0.80041045+1.0092261j , 0.94073445+0.9994043j ,
       1.1236668 +1.0416095j , 1.0656508 +1.0222751j ,
       0.94819057+1.0333216j , 0.91837484+0.98015106j,
       0.89652586+1.1199863j , 1.0369996 +0.9844777j ,
       0.96947694+0.8858478j , 0.92914003+1.1577988j ,
       1.1081312 +1.1393045j , 0.91411555+1.1345003j ,
       0.93595326+1.0770782j , 1.0835761 +0.9341351j ,
       1.0789803 +0.9809754j , 1.0851889 +0.9432453j ,
       0.9549229 +0.7950688j , 1.0810493 +0.94939154j,
       0.8450941 +1.0472014j , 0.77741945+0.9585086j ,
       0.94688606+1.1251614j , 1.0217459 +1.0306687j ,
       1.0300897 +1.018015j  , 0.82153714+0.9267591j ,
       0.9

In [29]:
cp.linalg.norm(r_src-p_dest)/cp.linalg.norm(r_src)

array(0.09284756, dtype=float32)

In [30]:
cp.linalg.norm(r_src-p_dest)/cp.linalg.norm(p_dest)

array(0.09325037, dtype=float32)

In [31]:
p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(r_src)))))))).flatten()[:50]

array([1.1924028 +0.9622046j , 1.131465  +0.9596402j ,
       1.055252  +0.82345235j, 1.0549535 +0.9251491j ,
       0.96159875+1.0393807j , 0.9530078 +1.0415289j ,
       0.9523366 +0.9495442j , 0.7833492 +1.0426009j ,
       1.1427467 +1.1257019j , 0.8673905 +1.0726188j ,
       0.78770274+1.0331659j , 0.8161577 +0.8263445j ,
       0.74181604+1.0979842j , 0.9804518 +0.8974335j ,
       0.9910274 +0.96278584j, 1.0458647 +1.0467169j ,
       1.0703461 +1.0456423j , 0.9062754 +0.971326j  ,
       1.0373149 +1.0902435j , 0.8214741 +1.0707424j ,
       0.866495  +0.89142364j, 0.95892066+0.9873811j ,
       0.8901612 +1.167287j  , 0.79725975+0.96232486j,
       1.0696646 +0.9613615j , 0.9215934 +1.081505j  ,
       0.9532185 +0.92094254j, 1.13575   +1.0021201j ,
       0.85560113+1.1228732j , 0.78500557+1.1069125j ,
       0.9278096 +0.97881943j, 1.028975  +1.0910052j ,
       1.0465842 +0.94756746j, 0.97705925+1.1155329j ,
       0.7421407 +1.0821455j , 1.2050409 +1.0633075j ,
       0.9

In [32]:
cp.linalg.norm(r_src-p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(r_src)))))))))/cp.linalg.norm(r_src) #???

array(0.09284756, dtype=float32)

In [33]:
# _mat = contract("escTtZzYyXx,escTtZzYyXx->scTtZzYyXx",
#                 testvectors, cp.conj(testvectors)).flatten()
# print(cp.linalg.norm(_mat))
# print(_mat[:100])

# MultiGrid - R*matvec\*P.

In [34]:
def _r_matvec_p(src, matvec):
    return r_vec(matvec(p_vec(io.xxx2eTZYX(src, params))))


def r_matvec_p(src, matvec):
    return io.array2xxx(_r_matvec_p(src, matvec))

# MultiGrid - verify above.

In [35]:
D_r_src = matvec(r_src)

multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001956951 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001927780 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.002077964 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001953575 sec


In [36]:
D_r_src.flatten()[:50]

array([ 0.09512203-0.15065098j,  0.02459466-0.10281426j,
        0.14802423-0.04478627j,  0.05378214-0.09193704j,
       -0.08474712+0.0208632j , -0.06649091+0.06668942j,
       -0.03820895+0.03143284j, -0.06110204+0.1998555j ,
       -0.09466349-0.10954653j, -0.09903856+0.15328293j,
       -0.11581516+0.23665299j,  0.15433274+0.0833789j ,
       -0.11228501+0.18325019j,  0.07416645+0.00099519j,
        0.05309803-0.03660914j,  0.034646  -0.03932991j,
       -0.00817882-0.03089109j,  0.03485045+0.13626698j,
       -0.12274602-0.06901661j, -0.10658678+0.15540114j,
        0.09019466+0.07479304j,  0.00329622+0.01262958j,
       -0.2735389 +0.08964469j, -0.02719212+0.18662325j,
        0.07507145-0.0618236j , -0.09112616+0.10951558j,
        0.08583128+0.02523299j, -0.0734987 -0.1731855j ,
       -0.09989857+0.04082112j, -0.1259279 +0.17906842j,
        0.00141903+0.04172344j, -0.08249421-0.07952813j,
        0.06730361-0.03395787j, -0.16046013+0.01053373j,
       -0.04409004+0.2682615j ,

In [37]:
p_r_D_p_r_dest=p_vec(_r_matvec_p(r_dest,matvec=cg_dslash))

Input Array Shape: (24, 8, 4, 4, 4)
Dest Shape: (24, 8, 4, 4, 4)
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.002039823 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001913949 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001937063 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001875627 sec


In [38]:
p_r_D_p_r_dest.flatten()[:50]

array([0.01284266+0.00043204j, 0.01207354+0.00220424j,
       0.0121465 +0.0025559j , 0.01484103+0.00671565j,
       0.00851426+0.0052227j , 0.00844077+0.00481056j,
       0.00867679+0.00409095j, 0.00811415+0.00531656j,
       0.00683777+0.00171679j, 0.0057756 +0.00181463j,
       0.00582205+0.00221079j, 0.00453556+0.00140342j,
       0.00929137+0.00846156j, 0.01103785+0.00450657j,
       0.01158802+0.00349668j, 0.0132275 +0.00333044j,
       0.01015243+0.00213161j, 0.00825791+0.00220426j,
       0.01114164+0.00498186j, 0.01093537+0.00712627j,
       0.00751002+0.00507913j, 0.00815575+0.00541701j,
       0.00889412+0.00674953j, 0.00828097+0.00466273j,
       0.00554528+0.00108429j, 0.00750764+0.00175549j,
       0.00710701+0.00027447j, 0.00992816+0.00175117j,
       0.01150915+0.01007911j, 0.00842286+0.00877829j,
       0.00990541+0.00592939j, 0.01135234+0.0060305j ,
       0.00789523+0.00191099j, 0.00920649+0.00426514j,
       0.00813269+0.00681124j, 0.0152209 +0.00934403j,
       0.0

In [39]:

cp.linalg.norm(D_r_src-p_r_D_p_r_dest)/cp.linalg.norm(D_r_src)

array(0.9956338, dtype=float32)

In [40]:
r_dest.flatten()[:50]

array([-26.228638-2.6980443j,  26.825924+2.2300706j,
       -27.157005-2.886311j ,  27.309052+2.1306527j,
        26.245262+3.1135411j,  26.968142+2.7365303j,
        27.338694+2.955482j ,  26.714493+2.8954268j,
       -26.562653-3.4426003j,  27.022251+3.223345j ,
        27.010738+2.8851447j,  26.904491+2.7066288j,
        26.563435+2.7313137j, -26.834703-2.6179743j,
        26.203568+2.8582335j,  26.13324 +2.7540636j,
       -26.757036-2.5445836j,  26.579767+2.9527426j,
        26.924713+2.852963j ,  26.084093+2.554791j ,
        26.118988+3.0851789j,  27.292454+2.5094728j,
        27.045673+2.9704423j,  25.87228 +2.7780027j,
        26.435436+3.0725877j,  26.982637+3.2305465j,
        26.135303+2.6885123j,  26.39829 +2.4337387j,
        26.226591+1.8348447j,  26.38916 +2.9997063j,
       -26.302986-2.5908453j, -26.14183 -1.8610659j,
        26.349154+3.348411j ,  26.792229+2.4964402j,
        26.288826+2.7947683j,  26.808006+2.8376613j,
        26.121277+3.1562634j,  27.091488+2.345

In [41]:
p_vec(r_dest).flatten()[:50]

array([1.1924026 +0.96220475j, 1.1314651 +0.9596404j ,
       1.0552521 +0.8234522j , 1.0549538 +0.92514884j,
       0.9615977 +1.0393802j , 0.95300674+1.0415285j ,
       0.9523357 +0.9495436j , 0.78334856+1.0426004j ,
       1.1427466 +1.1257019j , 0.8673905 +1.072619j  ,
       0.7877028 +1.033166j  , 0.81615794+0.8263445j ,
       0.7418158 +1.0979842j , 0.9804515 +0.89743346j,
       0.99102736+0.9627855j , 1.045865  +1.0467168j ,
       1.0703461 +1.0456425j , 0.90627563+0.97132605j,
       1.0373149 +1.0902432j , 0.82147443+1.0707421j ,
       0.86649394+0.89142334j, 0.95891976+0.98738074j,
       0.8901605 +1.1672864j , 0.79725903+0.9623244j ,
       1.0696645 +0.9613617j , 0.9215935 +1.0815051j ,
       0.9532187 +0.92094266j, 1.1357502 +1.0021203j ,
       0.85560083+1.1228731j , 0.78500533+1.1069124j ,
       0.92780966+0.97881925j, 1.028975  +1.0910051j ,
       1.0465842 +0.9475676j , 0.9770593 +1.115533j  ,
       0.74214095+1.0821452j , 1.2050415 +1.0633073j ,
       0.9

# MultiGrid - CG (BUG!!!)

In [42]:
# b_e = fermion_in[define._EVEN_].flatten()
# b_o = fermion_in[define._ODD_].flatten()
# b__o = cp.zeros_like(b_o)
# tmp = cp.zeros_like(b_o)
# # b__o=b_o+kappa*D_oe(b_e)
# qcu.applyWilsonDslashQcu(tmp, b_e, gauge, set_ptrs, wilson_dslash_oe_params)
# b__o = b_o+kappa*tmp
# # b__o -> Dslash^dag b__o
# b__o = cg_dslash_dag(b__o)

In [43]:
# # # Dslash(x_o)=b__o
# # cg_solver = cg.slover(b=b__o, matvec=cg_dslash, tol=1e-10, max_iter=1000000)
# # x_o = cg_solver.run()

# # mg version
# mg_b__o = r_vec(io.xxxtzyx2mg_xxxtzyx(io.fermion2sctzyx(b__o, params), params)).flatten()
# cg_solver = cg.slover(b=mg_b__o, matvec=functools.partial(r_matvec_p, matvec=cg_dslash),
#                       tol=1e-5, max_iter=1000000)
# mg_x_o = cg_solver.run()

In [44]:
# # x_e  =b_e+kappa*D_eo(x_o)
# qcu.applyWilsonDslashQcu(tmp, x_o, gauge, set_ptrs, wilson_dslash_eo_params)
# x_e = b_e+kappa*tmp
# # give qcu_fermion_out
# qcu_fermion_out = cp.zeros_like(quda_fermion_out)
# qcu_fermion_out[define._EVEN_] = x_e.reshape(
#     quda_fermion_out[define._EVEN_].shape)
# qcu_fermion_out[define._ODD_] = x_o.reshape(
#     quda_fermion_out[define._ODD_].shape)


In [45]:
# print(np.linalg.norm(qcu_fermion_out-quda_fermion_out) / \
#     np.linalg.norm(quda_fermion_out))

# MultiGrid - BISTABCG (TESTING......)

In [46]:
b_e = fermion_in[define._EVEN_].flatten()
b_o = fermion_in[define._ODD_].flatten()
b__o = cp.zeros_like(b_o)
tmp = cp.zeros_like(b_o)
# b__o=b_o+kappa*D_oe(b_e)
qcu.applyWilsonDslashQcu(tmp, b_e, gauge, set_ptrs, wilson_dslash_oe_params)
b__o = b_o+kappa*tmp

multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001933465 sec


In [47]:
# # Dslash(x_o)=b__o
# bistabcg_solver = bistabcg.slover(
#     b=b__o, matvec=bistabcg_dslash, tol=1e-10, max_iter=1000000)
# x_o = bistabcg_solver.run()
# io.xxx2hdf5_xxx(x_o, params, 'x_o.h5')

In [48]:
# mg version
mg_b__o = r_vec(io.xxxtzyx2mg_xxxtzyx(io.fermion2sctzyx(b__o, params), params)).flatten()
bistabcg_solver = bistabcg.slover(
    b=mg_b__o, matvec=functools.partial(r_matvec_p, matvec=bistabcg_dslash), tol=1e-10, max_iter=1000000)
mg_x_o = bistabcg_solver.run()
_x_o=io.array2xxx(p_vec(io.xxx2eTZYX(mg_x_o, params)))
io.xxx2hdf5_xxx(_x_o, params, '_x_o.h5')


Input Array Shape: (4, 3, 32, 32, 32, 16)
Dest Shape: (4, 3, 8, 4, 4, 8, 4, 8, 4, 4)
Input Array Shape: (12288,)
Dest Shape: (24, 8, 4, 4, 4)
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.058587040 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.002389440 sec
Input Array Shape: (12288,)
Dest Shape: (24, 8, 4, 4, 4)
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.044487860 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001934443 sec
Input Array Shape: (12288,)
Dest Shape: (24, 8, 4, 4, 4)
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.042151985 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001876398 sec
Iteration 0: Residual = 4.921270e+07, Time = 0.453735 s
Input Array Shape: (12288,)
Dest Shape: (24, 8, 4, 4, 4)
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.040916148 sec
multi-gpu wilson dslash total time: (without malloc free 

In [49]:
# # x_e  =b_e+kappa*D_eo(x_o)
# qcu.applyWilsonDslashQcu(tmp, x_o, gauge, set_ptrs, wilson_dslash_eo_params)
# x_e = b_e+kappa*tmp
# # give qcu_fermion_out
# qcu_fermion_out = cp.zeros_like(quda_fermion_out)
# qcu_fermion_out[define._EVEN_] = x_e.reshape(
#     quda_fermion_out[define._EVEN_].shape)
# qcu_fermion_out[define._ODD_] = x_o.reshape(
#     quda_fermion_out[define._ODD_].shape)
# print(np.linalg.norm(qcu_fermion_out-quda_fermion_out) / \
#     np.linalg.norm(quda_fermion_out))

In [50]:
x_o=io.hdf5_xxx2xxx(params,'x_o.h5')
_x_o=io.hdf5_xxx2xxx(params,'_x_o.h5')

Dest Shape: (6291456,)
Dest Shape: (6291456,)


In [51]:
x_o.flatten()[:50]

array([74.27604 +92.02284j , 75.06629 +87.020996j, 63.950462+81.42317j ,
       71.90431 +81.19496j , 82.32697 +77.19529j , 82.58721 +76.42146j ,
       75.92041 +76.61929j , 83.29001 +63.37289j , 90.26471 +89.01645j ,
       85.437584+67.62439j , 82.527954+61.06994j , 66.877235+63.152378j,
       86.607956+60.486282j, 69.97304 +78.853806j, 74.98956 +78.34286j ,
       82.30312 +82.18618j , 80.42873 +82.24485j , 75.13653 +69.27514j ,
       84.342064+79.84675j , 83.38979 +62.740913j, 70.93342 +69.10556j ,
       78.41749 +76.8612j  , 92.75483 +71.3723j  , 76.95801 +63.98175j ,
       76.83701 +84.04088j , 86.23326 +71.82796j , 74.12482 +74.47032j ,
       80.77271 +88.2747j  , 88.081   +68.75456j , 86.93835 +63.496307j,
       76.293976+74.005066j, 85.7808  +80.60354j , 73.14485 +80.3956j  ,
       86.552826+75.09205j , 84.15033 +56.783974j, 82.566315+92.946976j,
       80.47373 +77.690834j, 69.55831 +75.1817j  , 69.10493 +87.87933j ,
       78.504234+86.637924j, 85.1525  +73.68484j , 

In [52]:
_x_o.flatten()[:50]

array([69.09611 +55.615627j, 65.63428 +55.377068j, 61.209953+47.46849j ,
       61.200054+53.430695j, 55.786613+60.020485j, 55.31877 +59.8227j  ,
       55.255356+54.20688j , 45.406746+59.307213j, 66.09241 +65.454575j,
       50.321095+62.349075j, 45.750412+59.989754j, 47.744675+47.829746j,
       41.7406  +63.579193j, 55.58891 +52.30281j , 56.265125+56.056976j,
       59.485497+60.724777j, 62.022   +60.61207j , 52.5427  +56.155685j,
       60.19442 +62.95733j , 47.765327+61.928677j, 50.199554+51.55423j ,
       55.681786+56.70545j , 51.722824+66.80423j , 46.237396+54.728928j,
       61.780903+56.014095j, 53.314857+62.887474j, 55.25479 +53.478027j,
       66.03976 +57.903374j, 48.300972+64.90466j , 44.382202+64.08003j ,
       52.679733+56.829678j, 58.594902+63.25062j , 60.68333 +54.870785j,
       56.6784  +64.558876j, 43.17658 +62.53102j , 69.7999  +61.448906j,
       56.56581 +58.566902j, 54.23349 +49.67239j , 63.787704+48.662117j,
       62.533997+55.550785j, 54.343224+61.97721j , 

In [53]:
print(np.linalg.norm(_x_o-x_o) /
      np.linalg.norm(x_o))

0.28872454


# End for CG & BISTABCG. (pass, don't run this)

In [54]:
# cg_solver.end()
# bistabcg_solver.end()

# End for pyqcu. (pass, don't run this)

In [55]:
# qcu.applyEndQcu(set_ptrs, params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_eo_params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_oe_params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_eo_dag_params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_oe_dag_params)